In [1]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from utilsCPD import *
import matplotlib.pyplot as plt
from SWCPD import BaseDetector as SWDCP
import os
from pathlib import Path

In [4]:
file_name = 'Occupancy.csv'

file_path = os.path.normpath(os.path.join(os.getcwd(),"datasets/Occupancy",file_name))

Occupancy = pd.read_csv(file_path).iloc[:,1:]

Targets = pd.read_csv(os.path.normpath(os.path.join(os.getcwd(),"datasets/Occupancy/OccupancyTargets.csv"))).iloc[:,1:]

GroundTruth = list(Targets.to_numpy().flatten())

Occupancy.head()

,Temperature,Humidity,Light,CO2
0,23.18,27.2720,426.0,721.25
1,23.15,27.2675,429.5,714.00
2,23.15,27.2450,426.0,713.50
3,23.15,27.2000,426.0,708.25
4,23.10,27.2000,426.0,704.50


### SWCPD

In [ ]:
detector = SWDCP(data=Occupancy.to_numpy().astype(np.float32),window_length=500,max_history=500,significance=0.01)
print(detector.device)
detector.process_dataloader(p=2,n_theta=1000,seed=2025)
detector.evaluate(GroundTruth,30)

cuda


  4%|▍         | 336/7644 [00:04<01:38, 73.90it/s]

Change detected at: 828 
Initiate new segment


 14%|█▎        | 1050/7644 [00:17<01:49, 60.32it/s]

Change detected at: 1543 
Initiate new segment


...

100%|█████████▉| 7606/7644 [02:32<00:00, 50.13it/s]

Change detected at: 8100 
Initiate new segment


100%|██████████| 7644/7644 [02:32<00:00, 50.03it/s]

F1 score: 0.5833333333333334
Covering: 0.8032426077984104
0.5874125874125874


(0.5833333333333334, 0.8032426077984104, 0.5874125874125874)

### Clasp

In [4]:
from claspy.segmentation import BinaryClaSPSegmentation

clasp = BinaryClaSPSegmentation(window_size=50).fit(Occupancy.to_numpy())
print(f_measure({'0':GroundTruth},clasp.change_points,30))
covering({'0':GroundTruth},clasp.change_points,Occupancy.shape[0])

(0.2666666666666667, 0.5769230769230769)


0.19039153072090945

### Results for R methods

In [27]:
import json

#R=30,sig.lvl=0.05,min.size = 400
with open("./R/res/OccupancyECP.json") as f:
    res = json.load(f)[1:-1]

ECP_CPS =res
print("f1 & AUC",f_measure({'0':GroundTruth},ECP_CPS,30))
print("Covering",covering({'0':GroundTruth},ECP_CPS,Occupancy.shape[0]))


#BOCPD with datanormalization
with open("./R/res/OccupancyBOCPD.json") as f:
    res = json.load(f)[0][1:]

BOCPD_CPS =  res
print("F1 & AUC",f_measure({'0':GroundTruth},BOCPD_CPS,30))
print("Covering",covering({'0':GroundTruth},BOCPD_CPS,Occupancy.shape[0]))

f1 & AUC (0.33333333333333337, 0.3393665158371041)
Covering 0.6401316828686778
F1 & AUC (0.5454545454545455, 0.5711538461538461)
Covering 0.7367589017005471
